In [1]:
# update gdown, used to download stuff from google drive
!pip install -q --upgrade gdown

In [2]:
# download dataset
!gdown -q -O dataset.zip 1Mrx0OKnBFteOw1q8IZy-n8x9q8cxZwhT

In [3]:
# unzip dataset
!unzip -q -o dataset.zip

In [4]:
import pathlib
import shutil

import cv2
import numpy as np
import numpy.typing as npt
import tensorflow as tf

from loguru import logger
from skimage.transform import rotate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

DAEDALUS2_DIR = pathlib.Path("/workspaces/playground/playground/daedalus")

TRAIN_DATASET_DIR = DAEDALUS2_DIR / "post-processed"
TRAIN_DATASET_IMAGE_SIZE = (112, 112)
TRAIN_DATASET_CLASS_COUNT = 2996
TRAIN_BATCH_SIZE = 256

DATABASE_DIR = DAEDALUS2_DIR / "features_database"
MODEL_WEIGHTS_PATH = DAEDALUS2_DIR / "feature_extractor" / "weights"
FEATURE_VECTOR_SIZE = 64

MARQUINHO_TRAIN_IMAGE_PATH = DAEDALUS2_DIR / "marquinho_train.jpg"
MARQUINHO_TEST_IMAGE_PATH = DAEDALUS2_DIR / "marquinho_test.jpg"

RNG_SEED = 42

# ensure directories exist
assert TRAIN_DATASET_DIR.exists()
MODEL_WEIGHTS_PATH.parent.mkdir(parents=True, exist_ok=True)


In [5]:
def check_if_images_have_same_shape(
    dataset_dir: pathlib.Path = TRAIN_DATASET_DIR,
) -> None:
    paths = dataset_dir.rglob("*.jpg")
    imgs = [cv2.imread(str(p)) for p in paths]
    shapes = [img.shape for img in imgs]
    return np.all(np.asarray(shapes)), shapes[0]


# check_if_images_have_same_shape()


In [6]:
def load_dataset(
    dataset_dir: pathlib.Path = TRAIN_DATASET_DIR,
    rng_seed: int = RNG_SEED,
    batch_size: int = TRAIN_BATCH_SIZE,
) -> tf.data.Dataset:
    ds = tf.keras.utils.image_dataset_from_directory(
        directory=dataset_dir,
        batch_size=None,
        image_size=TRAIN_DATASET_IMAGE_SIZE,
        label_mode="categorical",
    )

    return (
        ds.map(lambda d, t: (tf.keras.applications.resnet_v2.preprocess_input(d), t))
        .cache()
        .shuffle(
            buffer_size=ds.cardinality().numpy(),
            seed=rng_seed,
            reshuffle_each_iteration=True,
        )
        .batch(batch_size, drop_remainder=True)
        .prefetch(tf.data.AUTOTUNE)
    )


# load_dataset().element_spec


In [9]:
def create_models() -> tf.keras.Model:
    base = tf.keras.applications.ResNet50V2(
        weights="imagenet",
        input_shape=TRAIN_DATASET_IMAGE_SIZE + (3,),
        include_top=False,
    )

    for layer in base.layers:
        layer.trainable = False

    model_input = tf.keras.Input(
        shape=TRAIN_DATASET_IMAGE_SIZE + (3,),
        batch_size=TRAIN_BATCH_SIZE,
    )

    data_aug = tf.keras.layers.RandomFlip(mode="horizontal")(model_input)
    data_aug = tf.keras.layers.RandomRotation(factor=15.0 / 360)(data_aug)
    data_aug = tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1)(
        data_aug
    )

    # the arch is not particularly important
    flatten = tf.keras.layers.Flatten()(base(data_aug))
    dense1 = tf.keras.layers.Dense(512, activation="relu")(flatten)
    dense1 = tf.keras.layers.BatchNormalization()(dense1)
    dense2 = tf.keras.layers.Dense(256, activation="relu")(dense1)
    dense2 = tf.keras.layers.BatchNormalization()(dense2)

    # we want the values to be between 0, 1
    feature_extractor_out = tf.keras.layers.Dense(
        FEATURE_VECTOR_SIZE,
        activation="sigmoid",
    )(dense2)

    feature_extractor = tf.keras.Model(
        inputs=model_input,
        outputs=feature_extractor_out,
        name="feature_extractor",
    )

    classifier_out = tf.keras.layers.Dense(
        TRAIN_DATASET_CLASS_COUNT,
        "softmax",
    )(feature_extractor_out)
    classifier = tf.keras.Model(
        inputs=model_input,
        outputs=classifier_out,
        name="classifier",
    )

    classifier.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics="accuracy",
    )

    return feature_extractor, classifier


In [11]:
def load_or_create_feature_extractor(
    train_dataset_dir: pathlib.Path = TRAIN_DATASET_DIR,
    model_weights_path: pathlib.Path = MODEL_WEIGHTS_PATH,
) -> tf.keras.Model:
    feature_extractor, classifier = create_models()

    try:
        classifier.load_weights(model_weights_path).expect_partial()
    except tf.errors.NotFoundError:
        ds = load_dataset(dataset_dir=train_dataset_dir)
        classifier.fit(
            ds,
            epochs=999,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor="loss",
                    patience=6,
                    restore_best_weights=True,
                ),
            ],
        )
        classifier.save_weights(model_weights_path, save_format="tf")

    return feature_extractor


load_or_create_feature_extractor()


Found 12000 files belonging to 2996 classes.
Epoch 1/999
46/46 [==============================] - 6s 63ms/step - loss: 7.9914 - accuracy: 0.0023
Epoch 2/999
46/46 [==============================] - 3s 62ms/step - loss: 7.7861 - accuracy: 0.0045
Epoch 3/999
46/46 [==============================] - 3s 60ms/step - loss: 7.6600 - accuracy: 0.0062
Epoch 4/999
46/46 [==============================] - 3s 59ms/step - loss: 7.5450 - accuracy: 0.0082
Epoch 5/999
46/46 [==============================] - 3s 61ms/step - loss: 7.4177 - accuracy: 0.0106
Epoch 6/999
46/46 [==============================] - 3s 60ms/step - loss: 7.2956 - accuracy: 0.0127
Epoch 7/999
46/46 [==============================] - 3s 61ms/step - loss: 7.1693 - accuracy: 0.0166
Epoch 8/999
46/46 [==============================] - 3s 61ms/step - loss: 7.0522 - accuracy: 0.0221
Epoch 9/999
46/46 [==============================] - 3s 60ms/step - loss: 6.9332 - accuracy: 0.0278
Epoch 10/999
46/46 [==============================] - 3

In [12]:
def load_image_and_extract_features(
    image_path: pathlib.Path,
    feature_extractor: tf.keras.Model,
) -> npt.NDArray[np.float32]:
    img = cv2.imread(str(image_path))
    resized_image = cv2.resize(img, TRAIN_DATASET_IMAGE_SIZE)
    batched_image = resized_image.reshape(1, *resized_image.shape)
    return feature_extractor.predict(batched_image).flatten()
    

In [23]:
def augment_image_for_database_insertion(
    original: npt.NDArray[np.float32]
) -> list[npt.NDArray[np.float32]]:
    """
    Returns a list containing the original image and augmentations consisting of flips and rotations.
    """

    flipped = [cv2.flip(original, 0), original]

    rotated_5 = [rotate(img, 5) for img in flipped]
    rotated_10 = [rotate(img, 10) for img in flipped]
    rotated_15 = [rotate(img, 15) for img in flipped]

    # add translation?

    return flipped + rotated_5 + rotated_10 + rotated_15



In [24]:
def add_new_instance_to_database(
    image_path: pathlib.Path,
    instance_label: str,
    database_dir: pathlib.Path,
    feature_extractor: tf.keras.Model,
) -> None:
    """
    `instance_path`: path to the image to be added to the database
    `instance_label`: the name or identifier of the the instance
    `database_dir`: location of the database
    `feature_extractor`: a pre-trained neural network that generated feature vectors
    """

    logger.info(f"storing new instance, label={instance_label}, path={image_path}")

    feature_vector = load_image_and_extract_features(
        image_path=image_path,
        feature_extractor=feature_extractor,
    )

    # create, if necessary, the label dir
    label_dir = database_dir / instance_label
    label_dir.mkdir(parents=True, exist_ok=True)

    # store the image on the database
    stored_image_path = label_dir / image_path.name

    if stored_image_path.exists():
        logger.warning(
            "there is already an instance with this filename in the database, overwriting"
        )

    # should copy image too?
    # stored_image_path.write_bytes(image_path.read_bytes())
    shutil.copy(src=image_path, dst=stored_image_path)

    # store feature vector on the database
    feature_vector_path = stored_image_path.with_suffix(".npy")
    np.save(
        file=feature_vector_path,
        arr=feature_vector,
    )


In [14]:
def populate_database_with_train_dataset(
    feature_extractor: tf.keras.Model,
    train_dataset_dir: pathlib.Path = TRAIN_DATASET_DIR,
    database_dir: pathlib.Path = DATABASE_DIR,
) -> None:
    for label_dir in train_dataset_dir.iterdir():
        for image_path in label_dir.iterdir():
            add_new_instance_to_database(
                image_path=image_path,
                instance_label=label_dir.name,
                database_dir=database_dir,
                feature_extractor=feature_extractor,
            )

populate_database_with_train_dataset(
    feature_extractor=load_or_create_feature_extractor()
)


2023-03-09 18:23:59.799 | INFO     | __main__:add_new_instance_to_database:33 - storing new instance, label=Vitali_Klitschko, path=/workspaces/playground/playground/daedalus/post-processed/Vitali_Klitschko/Vitali_Klitschko_0003_0001.jpg
2023-03-09 18:24:00.646 | INFO     | __main__:add_new_instance_to_database:33 - storing new instance, label=Vitali_Klitschko, path=/workspaces/playground/playground/daedalus/post-processed/Vitali_Klitschko/Vitali_Klitschko_0003_0000.jpg
2023-03-09 18:24:00.737 | INFO     | __main__:add_new_instance_to_database:33 - storing new instance, label=Vitali_Klitschko, path=/workspaces/playground/playground/daedalus/post-processed/Vitali_Klitschko/Vitali_Klitschko_0001_0001.jpg
2023-03-09 18:24:00.811 | INFO     | __main__:add_new_instance_to_database:33 - storing new instance, label=Vitali_Klitschko, path=/workspaces/playground/playground/daedalus/post-processed/Vitali_Klitschko/Vitali_Klitschko_0003_0002.jpg
2023-03-09 18:24:00.879 | INFO     | __main__:add_ne

In [15]:
def create_label_encoder(database_dir: pathlib.Path = DATABASE_DIR) -> LabelEncoder:
    label_dirs = sorted(database_dir.iterdir())
    label_names = [path.name for path in label_dirs]
    return LabelEncoder().fit(label_names)


In [16]:
def load_feature_vectors_and_labels(
    database_dir: pathlib.Path = DATABASE_DIR,
) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.int64]]:
    paths = list(database_dir.rglob("*.npy"))

    features = np.array([np.load(p) for p in paths])

    label_encoder = create_label_encoder(database_dir)
    labels = [p.parent.name for p in paths]
    encoded_labels = label_encoder.transform(labels)
    
    return features, encoded_labels

def _check_load_feature_vectors_and_labels() -> None:
    features, labels = load_feature_vectors_and_labels()
    print(features.shape, features.dtype)
    print(labels.shape, labels.dtype)

_check_load_feature_vectors_and_labels()

(12000, 64) float32
(12000,) int64


In [26]:
def classify_instance(
    instance_path: pathlib.Path,
    feature_extractor: tf.keras.Model,
) -> int:
    database_features, database_labels = load_feature_vectors_and_labels()
    model = KNeighborsClassifier(1).fit(database_features, database_labels)

    img = cv2.imread(str(instance_path))
    resized_img = cv2.resize(img, TRAIN_DATASET_IMAGE_SIZE)
    batched_image = resized_img.reshape(1, *resized_img.shape)
    instance_features = feature_extractor.predict(batched_image).flatten()

    return model.predict([instance_features])

In [18]:
!gdown -q -O marquinho_train.jpg "1EgvzTNEWTXvegURlmJAt8OXOtrKAlQEb"
!gdown -q -O marquinho_test.jpg "1RcLasSJj-XMke5Fj33adiaHWbbl-9ihW"

In [19]:
# show marqinho

In [27]:
add_new_instance_to_database(
    image_path=MARQUINHO_TRAIN_IMAGE_PATH,
    instance_label="marquinho",
    database_dir=DATABASE_DIR,
    feature_extractor=load_or_create_feature_extractor(),
    augment=True,
)

expeced_label = create_label_encoder().transform(["marquinho"])
predicted_label = classify_instance(MARQUINHO_TEST_IMAGE_PATH, load_or_create_feature_extractor())

print(expeced_label, predicted_label)

2023-03-09 18:41:15.421 | INFO     | __main__:add_new_instance_to_database:15 - storing new instance, label=marquinho, path=/workspaces/playground/playground/daedalus/marquinho_train.jpg


[2996] [517]
